In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, LogisticRegressionCV, Lasso, Ridge, RidgeClassifier, SGDClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, precision_recall_curve, precision_recall_fscore_support, f1_score, r2_score 
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint as sp_randint, gamma as sp_gamma, expon as sp_expon, uniform as sp_uniform
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
import cPickle as pickle

In [2]:
def open_prepper(file_path):
    """Open the DataPrepper from pickled file."""
    with open(file_path) as f:
        prepper = pickle.load(f)
    return prepper

file_path = '../data/store/data_prepper_ALL-CATEGORIES.pkl'
prepper = open_prepper(file_path)

X_train, y_train = prepper.return_training_data()
X_test, y_test = prepper.return_testing_data()

### IMAGE VIEW QUANTILES

In [5]:
model_ivq_GBC = GradientBoostingClassifier(loss='deviance', learning_rate=0.2793, n_estimators=500, subsample=0.878,
                                           min_samples_split=10, min_samples_leaf=10, min_weight_fraction_leaf=0.0,
                                           max_depth=5, init=None, random_state=None, max_features='auto', verbose=1,
                                           max_leaf_nodes=12, warm_start=True, presort='auto')

In [6]:
model_ivq_GBC.fit(X_train, y_train['image_views_quantized'])

      Iter       Train Loss      OOB Improve   Remaining Time 
         1      175577.5086        1545.6529         1117.02m
         2      169624.4471         834.6579         1120.62m
         3      165508.1553         532.6932         1098.15m
         4      162650.7832         367.7260         1096.96m
         5      160505.9044         279.5297         1100.38m
         6      158584.9172         252.0923         1103.86m
         7      157068.6581         172.5057         1101.86m
         8      156045.1083         120.6247         1095.72m
         9      154925.5663         114.5406         1090.68m
        10      154021.7192         119.5260         1084.94m
        20      147199.8696          35.3135         1047.69m
        30      143770.3274          17.7463          991.35m
        40      140833.7218           3.2083          936.78m
        50      138466.6431           1.8888          891.19m
        60      136399.5583           5.6201          850.81m
       

GradientBoostingClassifier(init=None, learning_rate=0.2793, loss='deviance',
              max_depth=5, max_features='auto', max_leaf_nodes=12,
              min_samples_leaf=10, min_samples_split=10,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              presort='auto', random_state=None, subsample=0.878,
              verbose=1, warm_start=True)

In [ ]:
random_search_GBC.best_score_

In [ ]:
best_GBC = random_search_GBC.best_estimator_
y_pred = best_GBC.predict(X_test)
print "Best AdaBoost DT F1 Score: ", f1_score(y_test['image_views_quantized'], y_pred, labels=None, pos_label=None, average='weighted', sample_weight=None)

In [ ]:
new_params = best_GBC.get_params()
new_params['n_estimators'] = 1000
new_params['verbose'] = 2
new_model_ivq_GBC = GradientBoostingClassifier(**new_params)
new_model_ivq_GBC.fit(X_train, y_train['image_views_quantized'])

In [ ]:
new_model_ivq_GBC.get_params()

In [ ]:
f1_scores = []
num_estimators = new_model_ivq_GBC.get_params()['n_estimators']
for i, y_pred in zip(range(1, num_estimators+1), new_model_ivq_GBC.staged_predict(X_test)):
    f1_scores.append(f1_score(y_test['image_views_quantized'], y_pred, labels=None, pos_label=None, average='weighted', sample_weight=None))

plt.plot(f1_scores, 'b')